In [76]:
class TreeNode:
    def __init__(self, data):
        self.data = data
        self.children = []
        

In [77]:
class PriorityQueueNode:
    def __init__(self,ele,priority):
        self.ele = ele
        self.priority = priority
        
class PriorityQueue:
    def __init__(self):
        self.pq = []
    
    def isEmpty(self):
        return self.getSize() == 0
    
    def getSize(self):
        return len(self.pq)

    def getMin(self):
        if self.isEmpty():
            return None
        return self.pq[0].ele
    
    def __percolateUp(self):
        childIndex = self.getSize() - 1
        while childIndex > 0:
            parentIndex = (childIndex-1)//2
            
            if self.pq[parentIndex].priority > self.pq[childIndex].priority:
                self.pq[parentIndex],self.pq[childIndex] = self.pq[childIndex],self.pq[parentIndex]
                childIndex = parentIndex
            else:
                break
            
    def insert(self,ele,priority):
        pqNode = PriorityQueueNode(ele,priority)
        self.pq.append(pqNode)
        self.__percolateUp()
        
    def removeMin(self):
        temp = self.pq[0]
        self.pq[0], self.pq[-1] = self.pq[-1], self.pq[0]
        self.pq.pop()
        parent = 0
        while parent < self.getSize() - 1:
            childl = 2*parent + 1
            childr = 2*parent + 2
            if childl < self.getSize() - 1:
                if self.pq[childl].priority < self.pq[parent].priority:
                    self.pq[childl], self.pq[parent] = self.pq[parent], self.pq[childl]
                    parent = childl

            elif childr < self.getSize() - 1:
                if self.pq[childr].priority > self.pq[parent].priority:
                    self.pq[childr], self.pq[parent] = self.pq[parent], self.pq[childr]
                    parent = childr
            else:
                break
        return temp
                

In [90]:
class Huff:
    def __init__(self, text):
        self.frequency = PriorityQueue()
        self.text = text
        
        
    def makefrequencyqueue(self):
        temp = dict()
        for character in self.text:
            temp[character]= temp.get(character, 0) + 1
            
        for key in temp.keys():
            self.frequency.insert(key, temp[key])
            
    def buildtree(self):
        while not self.frequency.isEmpty():
            x1 = self.frequency.removeMin()
            x2 = self.frequency.removeMin()
            n1 = TreeNode(x1)
            n2 = TreeNode(x2)
            newpriority = x1.priority + x2.priority
            parentNode = TreeNode(newpriority)
            parentNode.children.append(n1)
            parentNode.children.append(n2)
            if self.frequency.isEmpty():
                return parentNode
            self.frequency.insert(newpriority, newpriority)
            
            
def printTreedetailed(root):
    if not root: # edge case
        return
    print(root.data, ":", end="")
    for child in root.children:
        print(child.data, ",", end="")
    print()
    
    for child in root.children:
        printTreedetailed(child)
            

        
        

In [91]:
x = Huff("aaabbcdefghijkkkkklimdijejnfenfoiieofi")


In [92]:
x.makefrequencyqueue()

In [93]:
mytree = x.buildtree()

KeyboardInterrupt: 

In [ ]:
printTreedetailed(mytree)